# Enrichment analysis of TF dosage RNA data

In [1]:
suppressWarnings({
    library(clusterProfiler)
    library(org.Hs.eg.db)
    library("biomaRt")
    library(tidyverse)
#    library(diffEnrich)
    library(enrichR)
})



Registered S3 method overwritten by 'ggtree':
  method      from 
  identify.gg ggfun

clusterProfiler v4.2.2  For help: https://yulab-smu.top/biomedical-knowledge-mining-book/

If you use clusterProfiler in published research, please cite:
T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal enrichment tool for interpreting omics data. The Innovation. 2021, 2(3):100141


Attaching package: ‘clusterProfiler’


The following object is masked from ‘package:stats’:

    filter


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
 

### GO enrichment of DEGs (enrichR)

In [2]:
websiteLive <- getOption("enrichR.live")
if (websiteLive) {
    listEnrichrSites()
    setEnrichrSite("Enrichr") # Human genes   
}

Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is Live!

WormEnrichr ... 
Connection is Live!

YeastEnrichr ... 
Connection is Live!

FishEnrichr ... 
Connection is Live!

OxEnrichr ... 
Connection is Live!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



In [16]:
celltype <- "HEK293T"
suffix <- ""
group2 <- "GFP"
dbs <- c("GO_Molecular_Function_2015", "GO_Biological_Process_2015", "KEGG_2021_Human", "Reactome_2022")
indir <- sprintf("../../output/03-rna/01/tf_group/%s", paste0(celltype, suffix))
outdir <- sprintf("../../output/03-rna/03/tf_group/%s", paste0(celltype, suffix))
plotdir <- sprintf("../../plots/03-rna/03/tf_group/%s", paste0(celltype, suffix))

dir.create(outdir, recursive=T, showWarnings=F)
dir.create(plotdir, recursive=T, showWarnings=F)

tmp <- strsplit(Sys.glob(sprintf("%s/deseq_*_vs_%s_log2fc0.58_p0.05.txt", indir, group2)), split="deseq_|_vs") %>% as.data.frame %>% t
group1_ls <- tmp[,3] %>% unname

for (group1 in group1_ls){
    message(group1)
    df <- read.table(sprintf("%s/deseq_%s_vs_%s_log2fc0.58_p0.05.txt", indir, group1, group2), sep="\t")
    head(df)
    dim(df)
    
    for (direction in c("up", "down")){
        out_rds <- paste0(outdir, "/dotplot_go_", group1, "_vs_", group2, "_", direction, ".rds")
        if (direction=="up"){ # upregulated genes
            dff <- df[(df$padj<0.05)&(df$log2FoldChange >= 0.58),]
        } else{ # downregulated genes
            dff <- df[(df$padj<0.05)&(df$log2FoldChange <= -0.58),]
        }

        if (!file.exists(out_rds)){
            enriched <- enrichr(dff$gene_name, dbs)
            saveRDS(enriched, out_rds)
        } else{
            enriched <- readRDS(out_rds)
        }
        
        p <- list()
        for (i in seq_along(enriched)){
            res <- enriched[[i]]
            nterm <- 20
            p.cutoff <- 0.05
            sort_by <- "Combined.Score" # odds ratio * -ln(p)

            res$gene_count <- lapply(res$Overlap, function(n){strsplit(n, split="/")[[1]][1]}) %>% unlist %>% as.numeric
            #res <- res %>% dplyr::filter(Adjusted.P.value<p.cutoff) %>% dplyr::arrange(desc(Combined.Score))
            res <- res %>% dplyr::filter(P.value<p.cutoff) %>% dplyr::arrange(desc(Combined.Score))
            res$Term <- str_wrap(res$Term, width = 30)
            res$Term <- factor(res$Term, levels=rev(res$Term))

            p[[i]] <- ggplot(head(res, nterm), aes(x=Combined.Score, y=Term, color=P.value)) + geom_point(aes(size=gene_count)) + 
                xlab("Combined Score (Odds ratio x -ln(p))") +
                ggtitle(paste0(group1, " vs ", group2, " enrichment (", nrow(dff), " DEG genes)\n", names(enriched)[i])) +
                theme_minimal() + scale_color_viridis_c(direction = 1) + theme(panel.border = element_rect(colour = "black", fill=NA, size=0.5)) 
        }

        pdf(paste0(plotdir, "/dotplot_go_", group1, "_vs_", group2, "_", direction, "_p", p.cutoff,".pdf"), width=7, height=8)
        for (i in seq_along(p)){
            print(p[[i]])
        }
        dev.off()
    }
}


ALX4

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
BACH2

EF1a.KLF1

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
EF1a.SPI1

ELF1

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
FOXO1

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
FOXP1

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
FOXP3

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
GATA1

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
IKZF2

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted stri